Alot of this code comes from but is revised from
- nanogpt-lecture: [Andrej Karpathy, Github ng-video-lecture]( https://github.com/karpathy/ng-video-lecture?tab=readme-ov-file )

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
import pandas as pd
import data_process as dp

dp.pdb_to_3Di_csv("../data", "./data.csv")
df = pd.read_csv("./data.csv")

In [3]:
train, val = dp.get_train_val_split(df["3Di"].tolist(), ratio=0.9)
train, val

([[2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   12,
   2,
   12,
   12,
   2,
   2,
   12,
   2,
   2,
   2,
   12,
   12,
   12,
   12,
   2,
   2,
   17,
   12,
   12,
   2,
   12,
   12,
   12,
   12,
   12,
   12,
   17,
   12,
   12,
   12,
   17,
   17,
   12,
   12,
   12,
   17,
   12,
   12,
   12,
   12,
   2,
   2,
   12,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   12,
   12,
   2,
   12,
   12,
   12,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   2,
   2,
   12,
   17,
   11,
   17,
   17,
   17,
   17,
   17,
   17,
   13,
   13,
   17,
   17,
   1,
   17,
   17,
   17,
   17,
   6,
   2,
   15,
   12,
   12,
   15,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   12,
   12,
   12,
   

In [28]:
batch_size = 4
block_size = 8
X, Y = dp.get_batch(train, batch_size=batch_size, block_size=block_size)
print(X.shape, Y.shape) # (B, T)
print("X", X,"\nY", Y)

torch.Size([4, 8]) torch.Size([4, 8])
X tensor([[ 2,  2,  2,  2,  2, 12,  2, 12],
        [17, 17,  2,  6,  2,  9, 12,  2],
        [ 2,  2,  2, 12, 12,  2, 12, 12],
        [12, 12, 12, 12, 12, 12, 12,  2]]) 
Y tensor([[ 2,  2,  2,  2, 12,  2, 12, 12],
        [17,  2,  6,  2,  9, 12,  2,  9],
        [ 2,  2, 12, 12,  2, 12, 12,  2],
        [12, 12, 12, 12, 12, 12,  2,  2]])


In [127]:
# revised and copied from https://github.com/karpathy/ng-video-lecture?tab=readme-ov-file
class SimpleLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # (C, C)
    
    def forward(self, idx, targets=None):
        """idx is the context and is shape (B, T)"""

        # here we select out embeddings for all the tokens up to T
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            loss = None
        else:
            # generally speaking, this only happens during training
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) # each index is the target of each row in logits
            loss = F.cross_entropy(logits, targets)
        
        # logits is shape (B, T, C) at inference time and (B*T, C) at train
        return logits, loss
    

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
m = SimpleLM(len(dp.TOKENS))
logits, loss = m(X, Y)
print(logits.shape)
print(loss)

gen = m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=32)[0].tolist()
print(dp.decode(gen))

torch.Size([32, 20])
tensor(4.0602, grad_fn=<NllLossBackward0>)
ADLWADLTVNISNHGCTEWYAGMRGHCNGGTPF


convince myself I know what nn.Embedding does

In [128]:
a = nn.Embedding(20, 5)

In [129]:
# for each token in dimension T, select out a features row for it
print(a(torch.tensor([1,2,3,4])))

# can also select out per batch each vector to create (B, T, C) vector 
# where C is the embedding token dimension, T is the block size time and B is batches
print(a(torch.tensor([[1,2,3,4],
					  [0,0,0,0]])))
print(a(torch.tensor([[1,2,3,4],
					  [0,0,0,0]])).shape)

tensor([[-0.9128, -0.6759,  0.1348,  0.3194,  0.8161],
        [-0.1605,  0.3876,  1.5085, -1.0998,  0.6578],
        [-0.6419, -1.8890, -0.6116, -0.8736, -0.0471],
        [ 0.7036, -1.2988,  1.6406,  0.4195, -0.2187]],
       grad_fn=<EmbeddingBackward0>)
tensor([[[-0.9128, -0.6759,  0.1348,  0.3194,  0.8161],
         [-0.1605,  0.3876,  1.5085, -1.0998,  0.6578],
         [-0.6419, -1.8890, -0.6116, -0.8736, -0.0471],
         [ 0.7036, -1.2988,  1.6406,  0.4195, -0.2187]],

        [[-0.6824,  1.3332, -0.2112,  0.0955,  1.3360],
         [-0.6824,  1.3332, -0.2112,  0.0955,  1.3360],
         [-0.6824,  1.3332, -0.2112,  0.0955,  1.3360],
         [-0.6824,  1.3332, -0.2112,  0.0955,  1.3360]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([2, 4, 5])


Now start to implement language modeling

In [130]:
# now let's learn directly what the embeddings are for each token
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
block_size = 8
for steps in range(1000): # increase number of steps for good results...
    # sample a batch of data
    xb, yb = dp.get_batch(train, batch_size=batch_size, block_size=block_size)

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.466526746749878


In [109]:
gen = m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=128)[0].tolist()
print(dp.decode(gen))

ADDDATFHMNVPPPPQQDDSMEEPRQSQDDDPPWPPPKVVFNNVSVVNVHFEVVVDSRDPTTDPPQDHVLRQVFKRWQQLHVVHMWDSSNNTFKLHPWQDPDPPDPDPPPPPMWYVVFENVQLRDSRDC


In [110]:
class LayerNorm1d: 
  def __init__(self, dim, eps=1e-5):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    """mean 0 var 1 on each data point values (across T,C) and not on batch
    helps with larger training  
    """
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

In [4]:
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
vocab_size = 20
max_iters = 200
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 10
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = dp.get_batch(train if split == "train" else val, batch_size=batch_size, block_size=block_size, device=device)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class Block(nn.Module):
    """Attention Block that goes through Causal Multihead attention with an MLP at the end"""

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(
            vocab_size, n_embd
        )  # (vocab_size, n_embd) (n_embd is just Channels C)
        self.position_embedding_table = nn.Embedding(
            block_size, n_embd
        )  # so each character knows where it is in the sequence
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        """idx is the context and is shape (B, T)"""
        B, T = idx.shape

        # here we select out embeddings for all the tokens up to T and encode positions
        tok_embd = self.token_embedding_table(idx)  # (B, T, C)
        pos_embd = self.position_embedding_table(
            torch.arange(T, device=device)
        )  # select vectors based on context positions
        x = tok_embd + pos_embd

        # ATN 
        x = self.blocks(x)
        x = self.ln_f(x)

        # MLP at end
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            # generally speaking, this only happens during training
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(
                B * T
            )  # each index is the target of each row in logits
            loss = F.cross_entropy(logits, targets)

        # logits is shape (B, T, C) at inference time and (B*T, C) at train
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [5]:
model = GPT()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.203924 M parameters


In [6]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(m)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = dp.get_batch(train, batch_size=batch_size, block_size=block_size, device=device)

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 3.2599, val loss 3.2422
step 100: train loss 1.0557, val loss 1.3905
step 199: train loss 0.7823, val loss 1.5571


In [7]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(dp.decode(m.generate(context, max_new_tokens=128)[0].tolist()))

AVVLVLHDLWDSVLSVVVSVADPDPQADPVVVSVVVLLRPDDDDPDDDSVNVLTSNVLPDPVSCVSSVLCVSVVQPNVLCVLAPQLVLVLADDDDPGDPTLRPDSNSSNWDSVLADDNVLSVCVLRSVV
